In [2]:
# https://keras.io/
!pip install -q keras
import keras

Using TensorFlow backend.


In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam, SGD
from keras.preprocessing.image import ImageDataGenerator

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 128 #128 - original batch size
num_classes = 10
epochs = 50
l = 40
num_filter = 12
compression = 0.5
dropout_rate = 0.2

In [6]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]
#print (img_height, img_width, channel)

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

170500096/170498071 [==============================] - 15s 0us/step


In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_filter = 12  # original is 12 thinking of making iteration wiht 64
dropout_rate = 0.2
l = 12
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Fourth_Block = add_denseblock(Third_Transition, num_filter, dropout_rate)
Fourth_Transition = add_transition(Fourth_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Fourth_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [12]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 12)   324         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 12)   48          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 12)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=SGD(),
              metrics=['accuracy'])

In [0]:
# Adding Image Augmentation

from keras.preprocessing.image import ImageDataGenerator
gen = ImageDataGenerator(rotation_range=8, width_shift_range=0.08, shear_range=0.3,
                         height_shift_range=0.1, zoom_range=0.1)

test_gen = ImageDataGenerator()

train_generator = gen.flow(x_train, y_train, batch_size=batch_size)
test_generator = test_gen.flow(x_test, y_test, batch_size=batch_size)


model.fit_generator(train_generator, steps_per_epoch=60000//batch_size, epochs=epochs, validation_data=test_generator, validation_steps=10000//batch_size)

Epoch 1/50
468/468 [==============================] - 154s 328ms/step - loss: 1.1500 - acc: 0.5872 - val_loss: 1.7372 - val_acc: 0.5232
Epoch 2/50
468/468 [==============================] - 157s 336ms/step - loss: 1.1485 - acc: 0.5884 - val_loss: 1.2152 - val_acc: 0.6041
Epoch 3/50
468/468 [==============================] - 158s 337ms/step - loss: 1.1287 - acc: 0.5934 - val_loss: 1.3959 - val_acc: 0.5721
Epoch 4/50
468/468 [==============================] - 160s 342ms/step - loss: 1.1236 - acc: 0.5957 - val_loss: 1.2954 - val_acc: 0.5894
Epoch 5/50
468/468 [==============================] - 156s 333ms/step - loss: 1.1208 - acc: 0.5982 - val_loss: 1.1613 - val_acc: 0.6137
Epoch 6/50
467/468 [============================>.] - ETA: 0s - loss: 1.1125 - acc: 0.6003

In [14]:
# Model fit implimentation that came with original code
"""model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))"""

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 135s 3ms/step - loss: 2.2080 - acc: 0.1568 - val_loss: 2.1568 - val_acc: 0.1674
Epoch 2/50
50000/50000 [==============================] - 119s 2ms/step - loss: 1.9875 - acc: 0.2093 - val_loss: 1.9736 - val_acc: 0.2428
Epoch 3/50
50000/50000 [==============================] - 119s 2ms/step - loss: 1.8848 - acc: 0.2528 - val_loss: 2.0110 - val_acc: 0.2533
Epoch 4/50
50000/50000 [==============================] - 117s 2ms/step - loss: 1.8146 - acc: 0.2783 - val_loss: 1.8161 - val_acc: 0.2855
Epoch 5/50
50000/50000 [==============================] - 122s 2ms/step - loss: 1.7687 - acc: 0.2900 - val_loss: 1.7085 - val_acc: 0.3337
Epoch 6/50
50000/50000 [==============================] - 121s 2ms/step - loss: 1.7346 - acc: 0.3041 - val_loss: 1.9319 - val_acc: 0.2581
Epoch 7/50
50000/50000 [==============================] - 117s 2ms/step - loss: 1.7041 - acc: 0.3218 - val_loss: 1.6821 - 

In [13]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 12s 1ms/step
Test loss: 1.8637385402679443
Test accuracy: 0.444


In [14]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
from google.colab import files

files.download('DNST_model.h5')